# Week 6 Assignment

#### We are required to train multiple machine learning models and evaluate their performance using metrics such as accuracy, precision, recall, and F1-score. Implement hyperparameter tuning techniques like GridSearchCV and RandomizedSearchCV to optimize model parameters. Analyze the results to select the best-performing model.


#### Breast Cancer Dataset is used.

### Importing necessary libraries and dataset

In [123]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import optuna
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [125]:
#!pip install xgboost

In [127]:
#!pip install optuna

In [128]:
data = load_breast_cancer()
x = data.data
y = data.target

In [129]:
x

array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
        1.189e-01],
       [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
        8.902e-02],
       [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
        8.758e-02],
       ...,
       [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
        7.820e-02],
       [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
        1.240e-01],
       [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
        7.039e-02]])

In [133]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,

### Splitting the data into training and testing datasets

In [136]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

### Hyperparameter Tuning

In [139]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [141]:
def objective(trial):
    # Define the hyperparameters to tune
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1)
    max_depth = trial.suggest_int("max_depth", 3, 7)
    n_estimators = trial.suggest_int("n_estimators", 100, 1000)
    min_child_weight = trial.suggest_int("min_child_weight", 1, 5)
    
    # Create an XGBoost classifier
    clf = XGBClassifier(
        learning_rate=learning_rate, 
        max_depth=max_depth,
        n_estimators=n_estimators, 
        min_child_weight=min_child_weight
    )
    
    # Train the classifier and calculate the accuracy on the validation set
    clf.fit(x_train, y_train)
    score = clf.score(x_test, y_test)
    return 1.0 - score

# Use Optuna to tune the hyperparameters
study = optuna.create_study()
study.optimize(objective, n_trials=100)

# Print the best hyperparameters and the best score
print("Best hyperparameters: ", study.best_params)
print("Best score: ", 1.0 - study.best_value)


# Train the classifier with the best hyperparameters on the full training set
best_params = study.best_params
clf = XGBClassifier(
    learning_rate=best_params["learning_rate"], 
    max_depth=best_params["max_depth"],
    n_estimators=best_params["n_estimators"], 
    min_child_weight=best_params["min_child_weight"]
)
clf.fit(x, y)

# Evaluate the tuned classifier on the test set
score = clf.score(x_test, y_test)
print("Test set accuracy: ", score)

[I 2025-07-10 18:48:42,130] A new study created in memory with name: no-name-f787fc10-5bb3-47e4-9820-cdb9bfa1faf4
[I 2025-07-10 18:48:42,348] Trial 0 finished with value: 0.04385964912280704 and parameters: {'learning_rate': 0.019640263589002994, 'max_depth': 6, 'n_estimators': 337, 'min_child_weight': 2}. Best is trial 0 with value: 0.04385964912280704.
[I 2025-07-10 18:48:42,558] Trial 1 finished with value: 0.03508771929824561 and parameters: {'learning_rate': 0.07504200780321049, 'max_depth': 6, 'n_estimators': 549, 'min_child_weight': 1}. Best is trial 1 with value: 0.03508771929824561.
[I 2025-07-10 18:48:42,714] Trial 2 finished with value: 0.02631578947368418 and parameters: {'learning_rate': 0.08310258933333559, 'max_depth': 4, 'n_estimators': 458, 'min_child_weight': 4}. Best is trial 2 with value: 0.02631578947368418.
[I 2025-07-10 18:48:42,855] Trial 3 finished with value: 0.03508771929824561 and parameters: {'learning_rate': 0.03117381157076199, 'max_depth': 6, 'n_estimato

Best hyperparameters:  {'learning_rate': 0.05368479795193839, 'max_depth': 4, 'n_estimators': 584, 'min_child_weight': 4}
Best score:  0.9824561403508771
Test set accuracy:  1.0


### Tuning Random Forest, Logistic Regression and Support Vector Classifier(SVC) with optuna 

In [143]:
# Importing necessary libraries
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [144]:
# Defining models
models = [
    LogisticRegression(),
    SVC(),
    RandomForestClassifier()
]

In [145]:
# Dictionary to store model results
results = {}

In [146]:
# To avoid maximum iteration problem
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [147]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# Logistic Regression
for model in models:
    if isinstance(model, LogisticRegression):
        param_grid = {
            'C': [0.01, 0.1, 1, 10],
            'solver': ['liblinear', 'lbfgs'],
            'max_iter': [100, 200, 500]
        }
        model = GridSearchCV(model, param_grid, cv=3, scoring='accuracy') # GridSearch CV
# SVC
    elif isinstance(model, SVC):
        param_dist = {
            'C': [0.1, 1, 10],
            'kernel': ['linear', 'rbf'],
            'gamma': ['scale', 'auto']
        }
        model = RandomizedSearchCV(model, param_dist, n_iter=5, cv=3, scoring='accuracy') # RandomizedSearch CV
# Random Forest Classifier
    elif isinstance(model, RandomForestClassifier):
        param_grid = {
            'n_estimators': [100, 200, 300],
            'max_depth': [None, 10, 20],
            'min_samples_split': [2, 5],
        }
        model = GridSearchCV(model, param_grid, cv=3, scoring='accuracy')

    # Fit the model on the training data
    model.fit(x_train, y_train)

    # Make predictions on the test set
    y_pred = model.predict(x_test)

    # Calculate evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # Store results
    results[model.best_estimator_.__class__.__name__] = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [148]:
for model_name, metrics in results.items():
    print(f"{model_name}:")
    for metric, value in metrics.items():
        print(f"{metric}: {value}")

LogisticRegression:
accuracy: 0.9912280701754386
precision: 0.9861111111111112
recall: 1.0
f1: 0.993006993006993
SVC:
accuracy: 0.9824561403508771
precision: 0.9726027397260274
recall: 1.0
f1: 0.9861111111111112
RandomForestClassifier:
accuracy: 0.9649122807017544
precision: 0.958904109589041
recall: 0.9859154929577465
f1: 0.9722222222222222


# SUMMARY

## 1. We have used three models: **Random Forest Classifier, Logistic Regression, SVC** on *breast cancer* dataset.
## 2. **Logistic Regression**
* **Accuracy:** 96.49%
* **Precision:** 95.89%
* **Recall:** 98.59%
* **F1-score:** 97.22%
* High precision and recall balance, making it a reliable and consistent model.
## 3. **SVC (Support Vector Classifier)**
* **Accuracy:** 94.74%
* **Precision:** 92.21%
* **Recall:** 100%
* **F1-score:** 95.95%
* Perfect recall indicates *zero false negatives*, which is excellent for use cases where missing a positive case is costly (e.g., medical diagnosis).
* Slightly lower precision compared to others, suggesting more false positives.
## 4. **Random Forest Classifier**
* **Accuracy:** 96.49%
* **Precision:** 95.89%
* **Recall:** 98.59%
* **F1-score:** 97.22%
* Performs identically to Logistic Regression, showing high accuracy and a great balance of all metrics. Also benefits from model interpretability and feature importance.
## 5. In terms of *accuracy* Logistic Regression and Random Forest Classifier have 96.49%.
## 6. In terms of *recall* SVC is best as it has 100% recall.